In [ ]:
from Bayesian_Neural_Network import bayesian_neural_network
from BNN_Learning_Rate_Tuning import bnn_learning_rate_tuning

import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

np.random.seed(10120024)

# Data Preparation

In [ ]:
# load the dataset
df = pd.read_csv('sample_fraud.csv', 
                    usecols=['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud'], 
                    nrows=5000)

# perform one-hot-eencoding on the column 'type'
dummies_df = pd.get_dummies(df['type'], dtype=int)

# concat the original data with the one-hot-encoded data
data = pd.concat((df, dummies_df), axis=1) \
            .drop(columns=['type'])

In [ ]:
# df.to_csv('sample_fraud.csv', index=False)

In [ ]:
sample_data = pd.concat([
    data.loc[data['isFraud'] == 1, :],
    data.loc[data['isFraud'] == 0, :].head(500),
])

In [ ]:
sample_data

In [ ]:
# percentage of fraud and non-fraud label in the data
sample_data['isFraud'].value_counts().to_frame('percentage') / np.sum(sample_data['isFraud'].value_counts()) * 100

In [ ]:
# specify the feature and target data
feature_columns = sample_data.columns[data.columns != 'isFraud']
target_columns = ['isFraud']

feature_data = sample_data.loc[:, feature_columns]
target_data = sample_data.loc[:, target_columns]
target_data.loc[target_data['isFraud'] == 0, :] = -1

# standardize the dataset
for col in feature_columns[:5]:
    scaler = StandardScaler()
    feature_data[col] = scaler.fit_transform(feature_data[col].values.reshape(-1, 1))

In [ ]:
pca = PCA(n_components = 4)

feature_data_values = pca.fit_transform(feature_data.values)
feature_data = pd.DataFrame({f'PCA {i+1}':val for i, val in enumerate(feature_data_values.T)})
feature_columns = feature_data.columns

explained_variance = pca.explained_variance_ratio_

In [ ]:
np.sum(explained_variance)

In [ ]:
fraud_feature_data = feature_data.loc[(target_data['isFraud'] == 1).values, :].values
fraud_target_data = target_data.loc[(target_data['isFraud'] == 1).values, :].values 

non_fraud_feature_data = feature_data.loc[(target_data['isFraud'] == -1).values, :].values
non_fraud_target_data = target_data.loc[(target_data['isFraud'] == -1).values, :].values

fraud_non_fraud_feature_data = np.concatenate((fraud_feature_data, non_fraud_feature_data), axis=0)
fraud_non_fraud_target_data = np.concatenate((fraud_target_data, non_fraud_target_data), axis=0)

# Model Performance on Predicting Fraud Data (Labeled as 1)

In [ ]:
input_layer = [len(feature_columns)]
hidden_layers = [4, 8, 6, 8, 4]
output_layer = [1]

total_epochs = 50
initial_lr = 1e-2
end_lr = 1e-4
batch_size = 4

In [ ]:
bnn = bayesian_neural_network(input_layer, 
                                hidden_layers, 
                                output_layer, 
                                fraud_feature_data, 
                                fraud_target_data, 
                                batch_size=batch_size,
                                initial_lr=initial_lr, 
                                end_lr=end_lr,
                                total_epochs=total_epochs)

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == 1).T[0, 0]]])

In [ ]:
bnn.train_model()

In [ ]:
bnn.visualize_model_performance()

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == 1).T[0, 0]]])

# Model Performance on Predicting Non-Fraud Data (Labeled With -1)

In [ ]:
input_layer = [len(feature_columns)]
hidden_layers = [4, 8, 6, 8, 4]
output_layer = [1]

total_epochs = 100
initial_lr = 5e-3
end_lr = 1e-4
batch_size = 5

In [ ]:
bnn = bayesian_neural_network(input_layer, 
                                hidden_layers, 
                                output_layer, 
                                non_fraud_feature_data, 
                                non_fraud_target_data, 
                                batch_size=batch_size,
                                initial_lr=initial_lr, 
                                end_lr=end_lr,
                                total_epochs=total_epochs)

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == -1).T[0, 0]]])

In [ ]:
bnn.train_model()

In [ ]:
bnn.visualize_model_performance()

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == -1).T[0, 0]]])

# Model Performance on Fraud and Non-Fraud Data

In [ ]:
np.random.seed(10120024)

input_layer = [len(feature_columns)]
hidden_layers = [16, 18, 18, 16]
output_layer = [1]

total_epochs = 100
initial_lr = 1e-4
end_lr = 1e-8
batch_size = 40

In [ ]:
bnn = bayesian_neural_network(input_layer, 
                                hidden_layers, 
                                output_layer, 
                                fraud_non_fraud_feature_data, 
                                fraud_non_fraud_target_data, 
                                batch_size=batch_size,
                                initial_lr=initial_lr, 
                                end_lr=end_lr,
                                total_epochs=total_epochs)

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == 1).T[0, 0]]])

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == -1).T[0, 0]]])

In [ ]:
bnn.train_model()

In [ ]:
bnn.visualize_model_performance()

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == 1).T[0, 0]]])

In [ ]:
np.array([bnn.bnn_fp.forward_propagation(feature_data_i, bnn.m, bnn.v, bnn.model_structure)[-2][-1][0, 0] for feature_data_i in bnn.feature_data[(bnn.target_data == -1).T[0, 0]]])

In [ ]:
bnn.succesfull_train

# Model Learning Rate Tuning

In [ ]:
input_layer = [len(feature_columns)]
hidden_layers = [32, 16, 8, 4, 2]
output_layer = [1]

initial_lr_power = 20
end_lr_power = 24
total_iters = 4
tuning_epochs = 10

model_purpose = 'binary'

In [ ]:
bnn_lr_tuning = bnn_learning_rate_tuning(input_layer, 
                                         hidden_layers, 
                                         output_layer, 
                                         feature_data, 
                                         target_data,
                                         model_purpose=model_purpose, 
                                         initial_lr_power=initial_lr_power, 
                                         end_lr_power=end_lr_power, 
                                         total_iters=total_iters, 
                                         tuning_epochs=tuning_epochs)

In [ ]:
bnn_lr_tuning.perform_learning_rate_tuning()

In [ ]:
bnn_lr_tuning.visualize_learning_rate_tuning()